In [241]:
import pandas as pd
import json
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV


In [242]:
ptweets = []
for line in open('train_posi_tweets_2017.txt').readlines():
    ptweets.append(json.loads(line))

In [243]:
type(ptweets)

list

In [244]:
df = pd.DataFrame(ptweets)

In [245]:
df.head()

,Parent_embersId,city,coordinates,country,date,derivedFrom,embersId,events_words,instantloc,key_count,lang,score,screen_name,seed_words,text,text_items,uid
0,4d7d59b3bffc27d5fcc3517c1baba6934b408198,Sinaloa,NaN,Mexico,2012-07-04 11:35:15,1e1c5cc52bddab80e074483b2691ded6,a4d5fb7049c00ae6d1e054788214b4782b453262,NaN,"{'city': [['sinaloa', [-92.13, 15.89], 1055]],...",1.0,es,0.000715,ELDEBATE,NaN,#YoSoy132 protesta en Chiapas por resultado el...,"[[chiap, protest, electoral, result], [#yosoy1...",57719142.0
1,d28af3c7ab29669828430e97590f0b703869998b,Veracruz,NaN,Mexico,2012-07-06 15:53:48,1e1c782c6094ae00e07466896a285ee0,86e35566c836cc6cfde43a2d6621121b32b7ad81,NaN,"{'city': [['veracruz', [-96.14, 19.19], 559172...",1.0,es,0.001067,Agendamx,NaN,#Mexico #Veracruz #Xalapa Seguimos en Agenda c...,"[[segu, protest, charl, agend], [#mexico, #ver...",59896491.0
2,d63b75743eb69d7088963574ef6521d594d433a2,None,NaN,mexico,2012-12-01 21:30:53,1e23bfe62366a480e074c1c03b05ef6c,1a21034af71dbe01f02aca5191f56f90370cd4fc,NaN,"{'city': [], 'state': ['mexico']}",2.0,fr,3.126513,YaffaPRO,NaN,J'aime une vidéo @YouTube mise en ligne par @r...,"[[par, over, mexico, molotov, mise, presid, ai...",726480734.0
3,datasift:1e2a47a52674a080e074c89125d56340,Guerrero,"[-100.38, 28.31]",Mexico,2013-04-13 20:40:05,1e2a47a52674a080e074c89125d56340,4ae6e5a510b738a76e166a4c1d35443647e951a7,NaN,"{'city': [], 'state': ['guerrero']}",2.0,pt,0.003183,mexicoyy0,NaN,"""@ELNORTECOM: Protestan en Guerrero contra Gob...","[[gobern, protest, guerrer], [], [@elnortecom]...",769967058.0
4,datasift:1e2a437fe3b6ac80e074cdc3f29e8a02,NaN,NaN,mexico,2013-04-13 12:45:17,1e2a437fe3b6ac80e074cdc3f29e8a02,cca5417fb77db3254d290018fbec996fd9a27861,NaN,"{'city': [], 'state': []}",1.0,es,0.002903,yavendelo,NaN,"Maestros cierran comercios, bloquean ruta y el...","[[cierr, adviert, maestr, comerci, actu, prote...",62598443.0


In [246]:
a = [item[0] for item in df['text_items']]
df3 = pd.DataFrame()
df3['stemmed_words'] = a

In [247]:
df3.head()

,stemmed_words
0,"[chiap, protest, electoral, result]"
1,"[segu, protest, charl, agend]"
2,"[par, over, mexico, molotov, mise, presid, aim..."
3,"[gobern, protest, guerrer]"
4,"[cierr, adviert, maestr, comerci, actu, protes..."


In [248]:
b = [item[1] for item in df['text_items']]
df3['hashtags'] = b

In [249]:
df3.head()

,stemmed_words,hashtags
0,"[chiap, protest, electoral, result]",[#yosoy132]
1,"[segu, protest, charl, agend]","[#mexico, #veracruz, #xalapa]"
2,"[par, over, mexico, molotov, mise, presid, aim...",[]
3,"[gobern, protest, guerrer]",[]
4,"[cierr, adviert, maestr, comerci, actu, protes...",[]


In [250]:
df3['sentiment'] = 1

In [251]:
df3.head()

,stemmed_words,hashtags,sentiment
0,"[chiap, protest, electoral, result]",[#yosoy132],1
1,"[segu, protest, charl, agend]","[#mexico, #veracruz, #xalapa]",1
2,"[par, over, mexico, molotov, mise, presid, aim...",[],1
3,"[gobern, protest, guerrer]",[],1
4,"[cierr, adviert, maestr, comerci, actu, protes...",[],1


In [252]:
ntweets = []
for line in open('train_nega_tweets_2017.txt').readlines():
    ntweets.append(json.loads(line))

In [253]:
ndf = pd.DataFrame(ntweets)

In [254]:
ndf.head()

,Parent_embersId,city,coordinates,country,date,derivedFrom,embersId,events_words,instantloc,key_count,lang,score,screen_name,seed_words,text,text_items,uid
0,b66885f8c78726c20298ae951a3bdb4e41db8431,None,NaN,mexico,2012-10-10 04:28:17,1e21292e9b4ca680e074bd999ef8cc3a,163f1c928645369ad6687990b0767bb140664e51,NaN,"{'city': [], 'state': []}",1.0,es,0.006600,Manu_El_Suarez,NaN,@TheArkham No crea que me olvido de los amigos...,"[[olvid, march, abraz, amig, esper, bien, cre]...",82133994.0
1,2e9ca3d1dc1ca2808c10ad40f0d63d58e71f4741,Hernández,"[-102.04, 23.03]",Mexico,2013-01-16 21:46:39,1e2602635130a980e0744bad1c470046,b5834708e0047e5935c61cde4b6579c75cfdb5a1,NaN,"{'city': [], 'state': []}",3.0,es,0.001762,AdSch9,NaN,Avisale al IFE Que #SiDragonBallFueraMexicano ...,"[[dad, hernandez, noe, estup, ermitan, ife, se...",71342356.0
2,9759292f7a94ec22649a70abfe85cb9b276e93d5,None,NaN,mexico,2012-07-06 20:59:37,1e1c7ad7ee48a280e074090a50cfd5b8,06d7797785476868c4b58d44b78dc4ddab3da06c,NaN,"{'city': [], 'state': []}",1.0,es,0.001067,abogado_utopia,NaN,"@anabombon Ven, en España, a un Ebrard poco di...","[[obrador, espan, protest, dispuest, ven, ebra...",159296806.0
3,a93bc43bef3d7038abdbab1f14355a5e6f142fe6,None,NaN,mexico,2012-07-27 14:47:19,1e1d7f9f714fa580e074cc704794586a,3d3b6f3fdcc92a244e9dafa58da9dddda76d4d17,"[[radi, 2.6541338487006025]]",NaN,NaN,es,0.704868,SGUILLEN3,"[[mexic, 0.26557377049180325]]",El Mañanero - W Radio Mexico http://t.co/E9TTX...,"[[via, ver, imag, mananer, mariss, radi, ala, ...",148059299.0
4,a0b8626f160c71eab1d6b0f6a3f0a4e857f30fe4,Hernández,"[-102.04, 23.03]",Mexico,2013-01-16 21:05:35,1e26020786a7a980e074cc6865a2dffc,f539643c16d4a321c02460e71c74c895a2b2a4f5,NaN,"{'city': [], 'state': []}",3.0,-,0.001762,Changourbano,NaN,Parece que había cuentas pendientes entre Noé ...,"[[hernandez, vid, iron, cuent, noe, hab, ir, m...",114865269.0


In [255]:
a = [item[0] for item in ndf['text_items']]
b = [item[1] for item in ndf['text_items']]
c = 0

In [256]:
df4 = pd.DataFrame()
df4['stemmed_words'] = a
df4['hashtags'] = b
df4['sentiment'] = c

In [257]:
df4.head()

,stemmed_words,hashtags,sentiment
0,"[olvid, march, abraz, amig, esper, bien, cre]",[],0
1,"[dad, hernandez, noe, estup, ermitan, ife, sem...",[#sidragonballfueramexicano],0
2,"[obrador, espan, protest, dispuest, ven, ebrar...",[],0
3,"[via, ver, imag, mananer, mariss, radi, ala, n...",[],0
4,"[hernandez, vid, iron, cuent, noe, hab, ir, mu...",[],0


In [258]:
Train_Dataframe = df3.append(df4)

In [271]:
Train_Dataframe = Train_Dataframe.reset_index(drop=True)

In [306]:
nm = [item for item in Train_Dataframe['stemmed_words']]
y = [item for item in Train_Dataframe['sentiment']]

In [273]:
list_of_words = list()
for item in nm:
    for word in item:
        if(word):
            list_of_words.append(word)

In [319]:
list_of_words

['chiap',
 'protest',
 'electoral',
 'result',
 'segu',
 'protest',
 'charl',
 'agend',
 'par',
 'over',
 'mexico',
 'molotov',
 'mise',
 'presid',
 'aim',
 'protest',
 'video',
 'une',
 'in',
 'new',
 'adress',
 'lign',
 'inaugur',
 'gobern',
 'protest',
 'guerrer',
 'cierr',
 'adviert',
 'maestr',
 'comerci',
 'actu',
 'protest',
 'ces',
 'gobiern',
 'bloqu',
 'hast',
 'rut',
 'hemicicl',
 'apatr',
 'juarez',
 'malnac',
 'qpm',
 'feo',
 'pendej',
 'ninis',
 'dej',
 'vandal',
 'television',
 'ah',
 'epn',
 'protest',
 'bonit',
 'tom',
 'mir',
 'mexico',
 'amid',
 'in',
 'protest',
 'swear',
 'presid',
 'nadi',
 'superbowl',
 'qued',
 'viend',
 'cas',
 'sol',
 'protest',
 'put',
 'expresion',
 'via',
 'march',
 'inici',
 'bloqu',
 'mexic',
 'meg',
 'march',
 'angel',
 'may',
 'aguirr',
 'anunci',
 'moviliz',
 'inform',
 'gobiern',
 'ceteg',
 'doming',
 'march',
 'verbal',
 'person',
 'mism',
 'sex',
 'favor',
 'celay',
 'brev',
 'enfrent',
 'bod',
 'realment',
 'asegur',
 'infiltr',
 '

In [275]:
leng = len(list_of_words)

In [276]:
words = dict()
for i in range(leng):
    if len(list_of_words[i]) > 2:
        if list_of_words[i] in words:
            words[list_of_words[i]] = words[list_of_words[i]] + 1
        else:
            words[list_of_words[i]] = 1

In [277]:
words

{'chiap': 41,
 'protest': 2138,
 'electoral': 94,
 'result': 309,
 'segu': 112,
 'charl': 6,
 'agend': 7,
 'par': 84,
 'over': 34,
 'mexico': 383,
 'molotov': 13,
 'mise': 1,
 'presid': 97,
 'aim': 3,
 'video': 20,
 'une': 20,
 'new': 202,
 'adress': 1,
 'lign': 1,
 'inaugur': 23,
 'gobern': 56,
 'guerrer': 58,
 'cierr': 57,
 'adviert': 10,
 'maestr': 265,
 'comerci': 20,
 'actu': 19,
 'ces': 12,
 'gobiern': 201,
 'bloqu': 92,
 'hast': 2,
 'rut': 11,
 'hemicicl': 8,
 'apatr': 3,
 'juarez': 64,
 'malnac': 3,
 'qpm': 3,
 'feo': 15,
 'pendej': 65,
 'ninis': 5,
 'dej': 205,
 'vandal': 97,
 'television': 9,
 'epn': 203,
 'bonit': 33,
 'tom': 599,
 'mir': 51,
 'amid': 52,
 'swear': 42,
 'nadi': 57,
 'superbowl': 1,
 'qued': 121,
 'viend': 39,
 'cas': 158,
 'sol': 387,
 'put': 44,
 'expresion': 17,
 'via': 445,
 'march': 4274,
 'inici': 58,
 'mexic': 3359,
 'meg': 117,
 'angel': 152,
 'may': 85,
 'aguirr': 7,
 'anunci': 43,
 'moviliz': 12,
 'inform': 83,
 'ceteg': 41,
 'doming': 52,
 'verbal'

In [325]:
words = {list_of_words: freq for list_of_words, freq in words.items() if freq > 70}

In [333]:
words

{'acab': 77,
 'acuerd': 76,
 'against': 73,
 'ahor': 239,
 'algui': 73,
 'all': 74,
 'amig': 107,
 'amlo': 165,
 'amor': 93,
 'amp': 135,
 'analog': 104,
 'and': 304,
 'angel': 152,
 'anti': 91,
 'apagon': 119,
 'apoy': 150,
 'aqu': 72,
 'are': 107,
 'asi': 206,
 'atras': 142,
 'aun': 81,
 'autop': 94,
 'baj': 82,
 'bien': 218,
 'bloqu': 92,
 'buen': 255,
 'cad': 94,
 'call': 151,
 'cambi': 134,
 'can': 107,
 'cas': 158,
 'centr': 99,
 'chapultepec': 97,
 'city': 131,
 'ciud': 209,
 'ciudadan': 82,
 'com': 94,
 'convoc': 77,
 'cos': 112,
 'cre': 188,
 'cuent': 74,
 'dar': 86,
 'day': 72,
 'deb': 183,
 'dec': 116,
 'dej': 205,
 'derech': 106,
 'despu': 80,
 'dia': 270,
 'dias': 86,
 'dic': 156,
 'distrit': 110,
 'duel': 245,
 'educ': 72,
 'educacion': 75,
 'eleccion': 199,
 'elect': 78,
 'electoral': 94,
 'enriqu': 128,
 'epn': 203,
 'espan': 156,
 'esper': 209,
 'estan': 133,
 'estudi': 112,
 'exig': 91,
 'falt': 91,
 'favor': 186,
 'federal': 158,
 'for': 318,
 'fot': 109,
 'fraud': 1

In [334]:
words_idx = {list_of_words: idx for idx, (list_of_words, freq) in enumerate(words.items())}


In [335]:
X = []
Y = []

x = [0] * len(words)
for i in range(leng):
    if list_of_words[i] in words_idx:
        x[words_idx[list_of_words[i]]] += 1
X.append(x)


In [ ]:
svc = svm.SVC(kernel='linear')
Cs = range(1, 20)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs), cv = 10)
clf.fit(X, y)

In [ ]:
prediction = clf.predict(X)

In [339]:
ntweets = []
for line in open('test_tweets.txt').readlines():
    ntweets.append(json.loads(line))

In [340]:
dfd = pd.DataFrame(ntweets)

In [341]:
dfd.head()

,Parent_embersId,city,coordinates,country,date,derivedFrom,embersId,events_words,instantloc,key_count,lang,score,screen_name,seed_words,text,text_items,uid
0,b66885f8c78726c20298ae951a3bdb4e41db8431,None,NaN,mexico,2012-10-10 04:28:17,1e21292e9b4ca680e074bd999ef8cc3a,163f1c928645369ad6687990b0767bb140664e51,NaN,"{'city': [], 'state': []}",1.0,es,0.006600,Manu_El_Suarez,NaN,@TheArkham No crea que me olvido de los amigos...,"[[olvid, march, abraz, amig, esper, bien, cre]...",82133994.0
1,2e9ca3d1dc1ca2808c10ad40f0d63d58e71f4741,Hernández,"[-102.04, 23.03]",Mexico,2013-01-16 21:46:39,1e2602635130a980e0744bad1c470046,b5834708e0047e5935c61cde4b6579c75cfdb5a1,NaN,"{'city': [], 'state': []}",3.0,es,0.001762,AdSch9,NaN,Avisale al IFE Que #SiDragonBallFueraMexicano ...,"[[dad, hernandez, noe, estup, ermitan, ife, se...",71342356.0
2,9759292f7a94ec22649a70abfe85cb9b276e93d5,None,NaN,mexico,2012-07-06 20:59:37,1e1c7ad7ee48a280e074090a50cfd5b8,06d7797785476868c4b58d44b78dc4ddab3da06c,NaN,"{'city': [], 'state': []}",1.0,es,0.001067,abogado_utopia,NaN,"@anabombon Ven, en España, a un Ebrard poco di...","[[obrador, espan, protest, dispuest, ven, ebra...",159296806.0
3,a93bc43bef3d7038abdbab1f14355a5e6f142fe6,None,NaN,mexico,2012-07-27 14:47:19,1e1d7f9f714fa580e074cc704794586a,3d3b6f3fdcc92a244e9dafa58da9dddda76d4d17,"[[radi, 2.6541338487006025]]",NaN,NaN,es,0.704868,SGUILLEN3,"[[mexic, 0.26557377049180325]]",El Mañanero - W Radio Mexico http://t.co/E9TTX...,"[[via, ver, imag, mananer, mariss, radi, ala, ...",148059299.0
4,a0b8626f160c71eab1d6b0f6a3f0a4e857f30fe4,Hernández,"[-102.04, 23.03]",Mexico,2013-01-16 21:05:35,1e26020786a7a980e074cc6865a2dffc,f539643c16d4a321c02460e71c74c895a2b2a4f5,NaN,"{'city': [], 'state': []}",3.0,-,0.001762,Changourbano,NaN,Parece que había cuentas pendientes entre Noé ...,"[[hernandez, vid, iron, cuent, noe, hab, ir, m...",114865269.0


In [343]:
a = [item[0] for item in dfd['text_items']]
a

[['olvid', 'march', 'abraz', 'amig', 'esper', 'bien', 'cre'],
 ['dad',
  'hernandez',
  'noe',
  'estup',
  'ermitan',
  'ife',
  'semill',
  'qepd',
  'sid',
  'avisal'],
 ['obrador', 'espan', 'protest', 'dispuest', 'ven', 'ebrard', 'lopez', 'apoy'],
 ['via',
  'ver',
  'imag',
  'mananer',
  'mariss',
  'radi',
  'ala',
  'noo',
  'sra',
  'internet',
  'pued',
  'mexic'],
 ['hernandez',
  'vid',
  'iron',
  'cuent',
  'noe',
  'hab',
  'ir',
  'muert',
  'parec',
  'boom',
  'mejor',
  'pendient',
  'qepd'],
 ['lind',
  'tard',
  'labor',
  'exit',
  'gestion',
  'trabaj',
  'dur',
  'esfuerz',
  'asi',
  'cad'],
 ['city', 'right', 'is', 'whos', 'now', 'mexic'],
 ['on',
  'accus',
  'to',
  'mexico',
  'support',
  'reuter',
  'rule',
  'presid',
  'citi',
  'that',
  'thursday',
  'lent',
  'conserv',
  'their'],
 ['huelg', 'cuerp', 'cre'],
 ['dispon',
  'qued',
  'cavaret',
  'teatr',
  'list',
  'aun',
  'bolet',
  'manan',
  'plan',
  'lleg',
  'simpl',
  'estudi'],
 ['ass',
  '

[['olvid', 'march', 'abraz', 'amig', 'esper', 'bien', 'cre'],
 ['dad',
  'hernandez',
  'noe',
  'estup',
  'ermitan',
  'ife',
  'semill',
  'qepd',
  'sid',
  'avisal'],
 ['obrador', 'espan', 'protest', 'dispuest', 'ven', 'ebrard', 'lopez', 'apoy'],
 ['via',
  'ver',
  'imag',
  'mananer',
  'mariss',
  'radi',
  'ala',
  'noo',
  'sra',
  'internet',
  'pued',
  'mexic'],
 ['hernandez',
  'vid',
  'iron',
  'cuent',
  'noe',
  'hab',
  'ir',
  'muert',
  'parec',
  'boom',
  'mejor',
  'pendient',
  'qepd'],
 ['lind',
  'tard',
  'labor',
  'exit',
  'gestion',
  'trabaj',
  'dur',
  'esfuerz',
  'asi',
  'cad'],
 ['city', 'right', 'is', 'whos', 'now', 'mexic'],
 ['on',
  'accus',
  'to',
  'mexico',
  'support',
  'reuter',
  'rule',
  'presid',
  'citi',
  'that',
  'thursday',
  'lent',
  'conserv',
  'their'],
 ['huelg', 'cuerp', 'cre'],
 ['dispon',
  'qued',
  'cavaret',
  'teatr',
  'list',
  'aun',
  'bolet',
  'manan',
  'plan',
  'lleg',
  'simpl',
  'estudi'],
 ['ass',
  '

In [346]:
X = list()
for item in a:
    for word in item:
        if(word):
            X.append(word)

In [348]:
X

['olvid',
 'march',
 'abraz',
 'amig',
 'esper',
 'bien',
 'cre',
 'dad',
 'hernandez',
 'noe',
 'estup',
 'ermitan',
 'ife',
 'semill',
 'qepd',
 'sid',
 'avisal',
 'obrador',
 'espan',
 'protest',
 'dispuest',
 'ven',
 'ebrard',
 'lopez',
 'apoy',
 'via',
 'ver',
 'imag',
 'mananer',
 'mariss',
 'radi',
 'ala',
 'noo',
 'sra',
 'internet',
 'pued',
 'mexic',
 'hernandez',
 'vid',
 'iron',
 'cuent',
 'noe',
 'hab',
 'ir',
 'muert',
 'parec',
 'boom',
 'mejor',
 'pendient',
 'qepd',
 'lind',
 'tard',
 'labor',
 'exit',
 'gestion',
 'trabaj',
 'dur',
 'esfuerz',
 'asi',
 'cad',
 'city',
 'right',
 'is',
 'whos',
 'now',
 'mexic',
 'on',
 'accus',
 'to',
 'mexico',
 'support',
 'reuter',
 'rule',
 'presid',
 'citi',
 'that',
 'thursday',
 'lent',
 'conserv',
 'their',
 'huelg',
 'cuerp',
 'cre',
 'dispon',
 'qued',
 'cavaret',
 'teatr',
 'list',
 'aun',
 'bolet',
 'manan',
 'plan',
 'lleg',
 'simpl',
 'estudi',
 'ass',
 'and',
 'from',
 'to',
 'gav',
 'class',
 'when',
 'back',
 'an',
 '

In [ ]:
y = clf.predict(X)